<a href="https://colab.research.google.com/github/deimosfox/ML-ITMO-2021/blob/main/lab2/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). 

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;

In [ ]:
!pip install PyDrive
import pandas as pd
import numpy as np
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
downloaded = drive.CreateFile({'id':"1lEpoRKczv5EvZhff9O2I-0JkdHnbe_Mq"})
downloaded.GetContentFile('transactions.csv')
transactions = pd.read_csv('transactions.csv', sep = ',', nrows = 1000000)
downloaded = drive.CreateFile({'id':"1EP3KrATWS1I_qYdpRhYnSDl-eoBiOBQc"})
downloaded.GetContentFile('tr_types.csv')
tr_types = pd.read_csv('tr_types.csv', sep = ';')
downloaded = drive.CreateFile({'id':"10J8RzMIhoYHiad49r-oWNMAk-V5lo3OE"})
downloaded.GetContentFile('tr_mcc_codes.csv')
tr_mcc_codes = pd.read_csv('tr_mcc_codes.csv', sep = ';')
downloaded = drive.CreateFile({'id':"1FG1fopcmvMZ7GBaBOqQipccSeFoMUvNT"})
downloaded.GetContentFile('gender_train.csv')
gender_train = pd.read_csv('gender_train.csv', sep = ',')


Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'


In [89]:
smpl = transactions.sample(n=1000)
smpl_gr=smpl.groupby("tr_type").size()
correct_types = tr_types[tr_types["tr_description"].str.contains("АТМ|POS")]["tr_type"]
correct_types_in_smpl=correct_types[correct_types.isin(smpl_gr.index)].array
print(smpl_gr[correct_types_in_smpl].sum()/1000)

0.613


## Задание 2 


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [ ]:
top10 = transactions
top10['tr_type'].value_counts()

1010    231117
2010    151166
7070    149006
1110    137658
1030    118975
         ...  
4061         4
4096         4
8146         3
8100         2
1510         2
Name: tr_type, Length: 72, dtype: int64

In [ ]:
top10['tr_type'].value_counts()[:10].index.tolist()


[1010, 2010, 7070, 1110, 1030, 2370, 7010, 7030, 7071, 1100]

## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [ ]:
import math
smpl_inc = transactions.loc[transactions['amount'] > 0]
smpl_inc = smpl_inc[['customer_id','amount']]
smpl_outc = transactions.loc[transactions['amount'] <= 0]
smpl_outc = smpl_outc[['customer_id','amount']]
smpl_inc = smpl_inc.groupby('customer_id').sum()
smpl_outc = smpl_outc.groupby('customer_id').sum()

outcome = smpl_outc['amount'].min()
income = smpl_inc['amount'].max()
print(income, ': maximum income')
print(outcome, ': maximum outcome')

client_max_income = smpl_inc.nlargest(1, 'amount')
client_max_outcome = smpl_outc.nsmallest(1, 'amount')
smpl_inc = smpl_inc.sort_values(by=['amount'], ascending=False)
smpl_outc = smpl_outc.sort_values(by=['amount'])

client_max_income_id = smpl_inc.index[smpl_inc['amount'] == income].tolist()
print(client_max_income_id, ': id of maximum income')
client_max_outcome_id = smpl_outc.index[smpl_outc['amount'] == outcome].tolist()
print(client_max_outcome_id, ': id of maximum outcome')

int_1 = client_max_income_id[0]
int_2 = client_max_outcome_id[0]
smpl_grouped = transactions.groupby('customer_id').sum()

diff = math.fabs(income + outcome)
print(diff, ': difference')

1248114886.810022 : maximum income
-1249952204.789996 : maximum outcome
[70780820] : id of maximum income
[70780820] : id of maximum outcome
1837317.979973793 : difference


## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [ ]:
top10 = transactions['tr_type'].value_counts()
top10 = top10.sort_values(ascending=False).head(n=10)

for tr_type in top10.index:
  smpl = transactions[transactions['tr_type'] == tr_type]
  median = smpl['amount'].median()
  print(median, ': median for', tr_type, 'transaction type')
  mean = smpl['amount'].mean()
  print(mean, ': mean for', tr_type, 'transaction type')

-7411.52 : median for 1010 transaction type
-19784.748639648507 : mean for 1010 transaction type
-44918.32 : median for 2010 transaction type
-136077.62932505234 : mean for 2010 transaction type
11319.42 : median for 7070 transaction type
65569.83170018934 : mean for 7070 transaction type
-11207.57 : median for 1110 transaction type
-32119.330370770458 : mean for 1110 transaction type
-2245.92 : median for 1030 transaction type
-5320.980221811316 : mean for 1030 transaction type
-44918.32 : median for 2370 transaction type
-205418.24903167944 : mean for 2370 transaction type
112295.79 : median for 7010 transaction type
276391.7895961331 : mean for 7010 transaction type
13951.52 : median for 7030 transaction type
86104.33290896451 : mean for 7030 transaction type
3593.47 : median for 7071 transaction type
66806.82662251407 : mean for 7071 transaction type
-10188.259999999998 : median for 1100 transaction type
-44061.82726156657 : mean for 1100 transaction type


In [ ]:
income_client = transactions[transactions['customer_id'] == int_1]
outcome_client = transactions[transactions['customer_id'] == int_2]
income_types = income_client['tr_type'].unique()
outcome_types = outcome_client['tr_type'].unique()

print('Medians and means for income client:')
for tr_type in income_types:
  median = income_client[income_client['tr_type'] == tr_type]['amount'].median()
  print(median, ': median for', tr_type)
  mean = income_client[income_client['tr_type'] == tr_type]['amount'].mean()
  print(mean, ': mean for', tr_type)

print()

print('Medians and means for outcome client:')
for tr_type in outcome_types:
  median = outcome_client[outcome_client['tr_type'] == tr_type]['amount'].median()
  print(median, ': median for', tr_type)
  mean = outcome_client[outcome_client['tr_type'] == tr_type]['amount'].mean()
  print(mean, ': mean for', tr_type)

Medians and means for income client:
8848.91 : median for 7030
14061.955241583857 : mean for 7030
8714.15 : median for 7070
13848.205232082266 : mean for 7070
12543.439999999999 : median for 7071
18600.572007692284 : mean for 7071
10319.985 : median for 7020
14651.290741626804 : mean for 7020
12992.619999999999 : median for 7031
19210.358452380955 : mean for 7031
-2245915.77 : median for 2370
-2218504.987086618 : mean for 2370
-3368873.66 : median for 2010
-2941055.8709415505 : mean for 2010
10028.01 : median for 6110
17562.93173076922 : mean for 6110
9387.93 : median for 7074
22811.59307692308 : mean for 7074
14306.48 : median for 7034
14306.48 : mean for 7034
14396.32 : median for 7040
21538.332000000002 : mean for 7040
-2245915.77 : median for 2330
-2382398.344615385 : mean for 2330
28500.67 : median for 7021
28500.67 : mean for 7021

Medians and means for outcome client:
8848.91 : median for 7030
14061.955241583857 : mean for 7030
8714.15 : median for 7070
13848.205232082266 : mean

## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [ ]:
transactions = pd.merge(transactions, gender_train, how='left')
transactions = pd.merge(transactions, tr_mcc_codes, how='inner')
transactions = pd.merge(transactions, tr_types, how='inner')
transactions.shape

(999584, 9)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [ ]:
import math
women = transactions[(transactions['gender'] == 0) & (transactions['amount'] < 0)]
men = transactions[(transactions['gender'] == 1) & (transactions['amount'] < 0)]
women_mean = women['amount'].mean()
men_mean = men['amount'].mean()
print('module of mean (outcome):', math.fabs(math.fabs(women_mean) - math.fabs(men_mean)))

module of mean (outcome): 32718.05492021877


In [ ]:
import math
women = transactions[(transactions['gender'] == 0) & (transactions['amount'] > 0)]
men = transactions[(transactions['gender'] == 1) & (transactions['amount'] > 0)]
women_mean = women['amount'].mean()
men_mean = men['amount'].mean()
print('module of mean (income):', math.fabs(math.fabs(women_mean) - math.fabs(men_mean)))

module of mean (income): 63366.57104801515


## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income". 
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

In [ ]:

women_tr = pd.DataFrame([])
men_tr = pd.DataFrame([])
for tr_type in tr_types['tr_type']:
  women = transactions[(transactions['tr_type'] == tr_type) & (transactions['gender'] == 0) & (transactions['amount'] > 0)]
  men = transactions[(transactions['tr_type'] == tr_type) & (transactions['gender'] == 1) & (transactions['amount'] > 0)]
  if women.size > 0:
    women_index = women['amount'].idxmax()
    women_tr = women_tr.append(transactions.loc[women_index])
  if men.size > 0:
    men_index = men['amount'].idxmax()
    men_tr = men_tr.append(transactions.loc[men_index])
women_top10 = women_tr['amount'].sort_values().head(n=10)
men_top10 = men_tr['amount'].sort_values().head(n=10)

print('top10 minimum by max_income value for women:')
print(women_top10)
print()
print('top10 minimum by max_income value for men:')
men_top10

top10 minimum by max_income value for women:
782598     1122.96
715900     2245.92
713566     2245.92
752472     2335.75
718156     5041.86
998258     6737.75
714837    22459.16
321089    24839.83
669411    44918.32
716363    68315.82
Name: amount, dtype: float64

top10 minimum by max_income value for men:


713615      2245.92
802537      2245.92
386903     28803.87
518893     64682.37
999567     79280.83
998278     87590.72
638141     89836.63
714995    112295.79
755215    113194.15
718186    121279.45
Name: amount, dtype: float64

In [ ]:
women_top10 = women_tr.sort_values(by=['amount']).head(n=10)
men_top10 = men_tr.sort_values(by=['amount']).head(n=10)

women_types = women_top10['tr_type'].unique()
men_types = men_top10['tr_type'].unique()
same_types = pd.merge(women_top10, men_top10, on='tr_type', how='inner')
same_types[['amount_x','amount_y','tr_type','gender_x','gender_y']]

,amount_x,amount_y,tr_type,gender_x,gender_y
0,2245.92,2245.92,4110.0,0.0,1.0
1,2335.75,113194.15,2370.0,0.0,1.0
2,22459.16,112295.79,2110.0,0.0,1.0
3,24839.83,28803.87,1010.0,0.0,1.0
4,44918.32,89836.63,2010.0,0.0,1.0


In [ ]:
same_tr = same_types['tr_type'].unique()
res_tr = pd.DataFrame([])

for tr_type in same_tr:
  tr = transactions[transactions['tr_type'] == tr_type]
  res_tr = res_tr.append(tr)

#res_tr['mcc_code'].value_counts()
#res_tr = res_tr.sort_values(by = ['mcc_code'], ascending=False).head(n=5)
res_tr.groupby('mcc_code')
res_tr.head(n=5)

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,mcc_description,tr_description
713158,1585299,359 17:54:09,6011,4110,-2245.92,9428222,1.0,Финансовые институты — снятие наличности автом...,Плата за получение наличных в АТМ. Россия
713159,1585299,359 17:55:27,6011,4110,-2245.92,9428222,1.0,Финансовые институты — снятие наличности автом...,Плата за получение наличных в АТМ. Россия
713160,1585299,362 15:59:40,6011,4110,-2245.92,00004936,1.0,Финансовые институты — снятие наличности автом...,Плата за получение наличных в АТМ. Россия
713161,64627292,125 10:48:14,6011,4110,-2245.92,NaN,NaN,Финансовые институты — снятие наличности автом...,Плата за получение наличных в АТМ. Россия
713162,64627292,451 09:28:24,6011,4110,-2245.92,403540,NaN,Финансовые институты — снятие наличности автом...,Плата за получение наличных в АТМ. Россия


## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин. 
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов. 